In [6]:
%matplotlib inline
# import all shogun classes
from shogun import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
import time
import os

In [2]:
slope= 4.7

x_train = np.random.rand(100)*10
y_train = slope*(x_train)+np.random.randn(100)*5+2
y_true = slope*(x_train)+2
x_test = np.linspace(0,10, 40)
y_test = slope*(x_test)+2

shogun_x_train = RealFeatures(x_train.reshape(1,len(x_train)))
shogun_x_test = RealFeatures(x_test.reshape(1,len(x_test)))

In [8]:
etfDir = '/media/ak/My Passport/MarketData/ETF_Levels/'
etfSymbols = sorted(os.listdir(etfDir))
etfIdx=2 #index for etf file
print('Pick the', etfIdx, 'symbol to play with')
### combine symbol index with etfDir to get all the location
etfSymbolFiles = os.path.join(etfDir, etfSymbols[etfIdx])

print('Symbol Location:', etfSymbolFiles)
## and make a list of all the dates
listETFSymbolFiles = os.listdir(etfSymbolFiles)

print('Picked the following ETF: ',etfSymbols[etfIdx])

Pick the 2 symbol to play with
Symbol Location: /media/ak/My Passport/MarketData/ETF_Levels/EWJ
Picked the following ETF:  EWJ


In [9]:
## ETF 1
for etfIdx in range(0,5):
    etfSymbolFiles = os.path.join(etfDir, etfSymbols[etfIdx])

    print('Symbol Location:', etfSymbolFiles)
    ## and make a list of all the dates
    listETFSymbolFiles = os.listdir(etfSymbolFiles)

    print('Picked the following ETF: ',etfSymbols[etfIdx])
    

Symbol Location: /media/ak/My Passport/MarketData/ETF_Levels/EEM
Picked the following ETF:  EEM
Symbol Location: /media/ak/My Passport/MarketData/ETF_Levels/EFA
Picked the following ETF:  EFA
Symbol Location: /media/ak/My Passport/MarketData/ETF_Levels/EWJ
Picked the following ETF:  EWJ
Symbol Location: /media/ak/My Passport/MarketData/ETF_Levels/FXI
Picked the following ETF:  FXI
Symbol Location: /media/ak/My Passport/MarketData/ETF_Levels/GDX
Picked the following ETF:  GDX


In [11]:
for etf in etfSymbols:
    with open('{}.csv'.format(etf), 'a') as f_output:
        print('Picked the following ETF', file=f_output) 

In [ ]:
combinedETFfiles_A = dict() # combine all the files for a single ETF
combinedETFfiles_B = dict() # combine all the files for a single ETF


for fileIdx, _ in enumerate(os.listdir(etfSymbolFiles)):
    '''
    go through all the dates for the symbol choice above and create a location for each 
    etf symbol+ date csv and read it into a pandas dataframe then store it
    
    '''
    etfFileDate = os.listdir(etfSymbolFiles)[fileIdx]
    fileLoc = os.path.join(etfSymbolFiles, os.listdir(etfSymbolFiles)[fileIdx])
    combinedETFfiles[etfFileDate.split('.')[0]] = pd.read_csv(fileLoc)

In [2]:
#mmd = LinearTimeMMD()
kernel = GaussianKernel(10, 1)
mmd.set_kernel(kernel)
mmd.set_p(features_p)
mmd.set_q(features_q)
mmd.set_num_samples_p(1000)
mmd.set_num_samples_q(1000)
alpha = 0.05

NameError: name 'GaussianKernel' is not defined